# Datasets: Lead Titanate and Lead Zirconate Titanate (PTO and PZT)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pycroscopy/DTMicroscope/blob/main/data/PZT_Data.ipynb)


This notebook explains how to access several available datasets pertaining to ferroelectric thin films of PZT and PTO.

We will show you where to find the data, how to register it on the digital microscope, and how to explore it offline.

**This notebook will be continuously updated with more datasets in due course, before the hackathon begins.**

By: Rama Vasudevan



# 1. Run server

In [1]:
!pip install pyro5
!pip install scifireaders
!pip install sidpy
!pip install pynsid
!pip install git+https://github.com/pycroscopy/DTMicroscope.git


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Cloning https://github.com/pycroscopy/DTMicroscope.git to c:\users\aisci\appdata\local\temp\pip-req-build-47sp1pfi
  Resolved https://github.com/pycroscopy/DTMicroscope.git to commit 23f925cbaa141d958795736266a9246effd53532
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/pycroscopy/DTMicroscope.git 'C:\Users\aisci\AppData\Local\Temp\pip-req-build-47sp1pfi'

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!run_server_afm

Server started and running in the background. Logs are being written to server.log.


# 2. Client side

In [3]:
import matplotlib.pylab as plt
import numpy as np
import Pyro5.api
from IPython.display import clear_output, display

## 2.1 Download Imaging Data

We can clone the repository to get the data files associated with PZT/PTO. This dataset consists of different Piezoresponse Force Microscopy (PFM) Images

In [4]:
!git clone https://github.com/pycroscopy/DTMicroscope.git

import os
os.listdir('DTMicroscope/data/AFM') #list available images

Cloning into 'DTMicroscope'...
Updating files:  61% (56/91)
Updating files:  62% (57/91)
Updating files:  63% (58/91)
Updating files:  64% (59/91)
Updating files:  65% (60/91)
Updating files:  67% (61/91)
Updating files:  68% (62/91)
Updating files:  69% (63/91)
Updating files:  70% (64/91)
Updating files:  71% (65/91)
Updating files:  72% (66/91)
Updating files:  73% (67/91)
Updating files:  74% (68/91)
Updating files:  75% (69/91)
Updating files:  76% (70/91)
Updating files:  78% (71/91)
Updating files:  79% (72/91)
Updating files:  80% (73/91)
Updating files:  81% (74/91)
Updating files:  82% (75/91)
Updating files:  83% (76/91)
Updating files:  84% (77/91)
Updating files:  85% (78/91)
Updating files:  86% (79/91)
Updating files:  87% (80/91)
Updating files:  89% (81/91)
Updating files:  90% (82/91)
Updating files:  91% (83/91)
Updating files:  92% (84/91)
Updating files:  93% (85/91)
Updating files:  94% (86/91)
Updating files:  95% (87/91)
Updating files:  96% (88/91)
Updating fil

['PMN28Pt0000.h5',
 'PMN28Pt0001.h5',
 'PMN28Pt0002.h5',
 'PTO_Funakubo0001.h5',
 'PTO_Funakubo0002.h5',
 'PTO_Funakubo0003.h5',
 'PTO_Funakubo0004.h5',
 'PTO_Funakubo0005.h5',
 'PTO_Funakubo0006.h5',
 'PTO_Funakubo0007.h5',
 'PZT_final0001.h5',
 'PZT_final0002.h5',
 'PZT_final0003.h5',
 'PZT_thick0004.h5',
 'PZT_thick0006.h5',
 'PZT_thick0009.h5']

## 2.2 Register Data to the Digital twin

In [5]:
uri = "PYRO:microscope.server@localhost:9092" #port for the AFM DT 9092
mic_server = Pyro5.api.Proxy(uri)
import sys

if 'google.colab' in sys.modules:
    mic_server.initialize_microscope("AFM", data_path = r"DTMicroscope/data/AFM/PZT_final0001.h5")
else:
    mic_server.initialize_microscope("AFM", data_path = "../test/datasets/dset_spm1.h5")
mic_server.setup_microscope(data_source = 'Compound_Dataset_1')#upload dataset?
mic_server.get_dataset_info()

CommunicationError: cannot connect to ('localhost', 9092): [WinError 10061] No connection could be made because the target machine actively refused it

# 3. Basic AFM operations

## 3.1 .get_scan()

The `get_scan()` function retrieves scan data from the dataset, with the option to filter by specific channels. Use the `channels` attribute to specify desired channels, allowing you to query particular signals or data channels. For details on available channels, refer to `.get_dataset_info()`.

The `direction` parameter specifies the fast-scanning axis, while the `trace` parameter emulates data acquisition on either the forward or backward pass. These settings are essential for accurately simulating the behavior of the real PID during scanning.

The `modification` parameter allows the application of various imperfections to simulate realistic scanning conditions, such as the effects of a broken tip or non-ideal PID performance.

In [ ]:
#scan without modification
array_list, shape, dtype  =  mic_server.get_scan(channels=['HeightRetrace', 'Phase1Retrace'], 
                                                 modification=None,
                                                 direction = 'horizontal',
                                                 trace = 'forward')
dat =  np.array(array_list, dtype=dtype).reshape(shape)
dat.shape

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(7,3))

ax[0].imshow(dat[0].T, cmap='cividis', origin='lower')
ax[0].set_title('HeightRetrace')
ax[1].imshow(dat[1].T, cmap='cividis', origin='lower')
ax[1].set_title('Phase1Retrace')